In [ ]:
import sqlite3
import requests
from tqdm import tqdm

In [3]:

def check_and_remove_invalid_cover_links(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT rowid, cover FROM history")
    rows = cursor.fetchall()
    for rowid, cover_url in rows:
        if cover_url:
            try:
                response = requests.head(cover_url, timeout=5)
                if response.status_code >= 400:
                    cursor.execute("UPDATE history SET cover=NULL WHERE rowid=?", (rowid,))
            except Exception:
                cursor.execute("UPDATE history SET cover=NULL WHERE rowid=?", (rowid,))
    conn.commit()
    conn.close()

conn = sqlite3.connect("history.db")
cursor = conn.cursor()
cursor.execute("SELECT rowid, cover FROM history")
rows = cursor.fetchall()


In [ ]:

for rowid, cover_url in tqdm(rows, desc="Checking cover links"):
    if cover_url:
        try:
            response = requests.head(cover_url, timeout=5)
            if response.status_code >= 400:
                cursor.execute("UPDATE history SET cover=NULL WHERE rowid=?", (rowid,))
        except Exception:
            cursor.execute("UPDATE history SET cover=NULL WHERE rowid=?", (rowid,))
conn.commit()
conn.close()

In [4]:
conn = sqlite3.connect("history.db")
cursor = conn.cursor()
cursor.execute("UPDATE history SET cover='' WHERE cover IS NULL")
conn.commit()
conn.close()